In [18]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

modele = load_model("bilstm700000.h5")
csv_file = "dataQuestionnaire.csv"
data = pd.read_csv(csv_file)

In [19]:
print(data.shape)

(741, 90)


In [20]:
# Liste des colonnes avec des commentaires
texte_colonnes = [
    "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)",
    "Pouvez-vous expliquer vos réponses merci ..."
]

# Calcul des nombres de valeurs non nulles pour chaque colonne avec des commentaires
for colonne in texte_colonnes:
    nb_non_null = data[colonne].count()
    print(f"Nombre de valeurs non nulles dans la colonne '{colonne}': {nb_non_null}")


Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)': 104
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)': 183
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)': 185
Nombre de valeurs non nulles dans la colonne 'Pouvez-vous expliquer vos réponses merci ...': 366


In [21]:
# Fonction pour nettoyer le texte
def nettoyer_texte(texte):
    if isinstance(texte, str):
        texte = texte.lower()
        texte = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", texte)
        texte = re.sub(r'[@/#]', '', texte)
        texte = re.sub("(\\d|\\W)+", " ", texte)
        texte = re.sub(r'_+', '', texte)
        texte = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', texte)
        texte = re.sub(r'http\S+', '', texte)
        texte = re.sub(r'\s+', ' ', texte).strip()
    else:
        texte = str(texte)
    return texte

MAX_SEQUENCE_LENGTH = 100
seuil = 0.5

# Fonction pour prédire la note
def predire_note(commentaire):
    if isinstance(commentaire, str):
        try:
            commentaire_seq = tokenizer.texts_to_sequences([commentaire])
            commentaire_pad = pad_sequences(commentaire_seq, maxlen=MAX_SEQUENCE_LENGTH)
            note = modele.predict(commentaire_pad)[0][0]
            return note
        except:
            return np.nan
    else:
        return np.nan

In [22]:
colonnes = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)"]
personnelle = data[colonnes].copy()
personnelle = personnelle.dropna(subset=[colonnes[1]])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(personnelle[colonnes[1]])

personnelle[colonnes[1]] = personnelle[colonnes[1]].apply(nettoyer_texte)
personnelle["Note personnelle"] = personnelle[colonnes[1]].apply(predire_note)

print(personnelle.head())
print(personnelle["Note personnelle"].value_counts())

1/1 [==============================] - 0s 31ms/step
         SID  \
1   15896547   
10  15897993   
14  15899015   
15  15899810   
20  15901382   

   Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  \
1   il n y a plus d ufr à l université de bordeaux...                              
10                   temps moyen hors période bouchon                              
14  temps moyen de déplacement selon les sites peu...                              
15  je suis responsable de m mon travail a une for...                              
20  pas de distance en temps pour transiter du lie...                              

    Note personnelle  
1           0.720357  
10          0.753010  
14          0.491092  
15          0.785849  
20          0.431251  
0.671572    4
0.445929    3
0.502898    2
0.692881    1
0.433342    1
           ..
0.187374    1
0.690571    1
0.158401    1
0.648759    1
0.163930    1
Name: Note personnelle, Length: 98, dtype: int64


In [23]:
print(data.shape)

(741, 90)


In [24]:
colonnes2 = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)"]
enseignement = data[colonnes2].copy()
enseignement = enseignement.dropna(subset=[colonnes2[1]])

enseignement[colonnes2[1]] = enseignement[colonnes2[1]].apply(nettoyer_texte)
enseignement["Note enseignement"] = enseignement[colonnes2[1]].apply(predire_note)

print(enseignement["Note enseignement"].value_counts())

1/1 [==============================] - 0s 67ms/step
0.345819    1
0.271843    1
0.379389    1
0.671572    1
0.596580    1
           ..
0.383593    1
0.433163    1
0.593593    1
0.387569    1
0.310462    1
Name: Note enseignement, Length: 183, dtype: int64


In [25]:
enseignement.head(20)

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement
4,15896653,l enseignement est de plus en plus difficile j...,0.345819
10,15897993,a ma connaissance mon enseignement n est pas é...,0.749795
13,15898762,attention pour être reconnu l investissement p...,0.522412
14,15899015,dernière question s il y a et non pas s il n y...,0.581034
15,15899810,quand un jeune me demande si devenir prof de f...,0.096319
18,15900109,ce n est pas qu un probleme de nb d heures mai...,0.251765
25,15902841,je bénéficie d une décharge financer sur proje...,0.816439
34,15903256,appréciation personnelle aucune option ne me c...,0.654708
35,15903369,la très forte augmentation du nombre d étudian...,0.533071
43,15904144,j aime toujours enseigner mais un simple servi...,0.569486


In [26]:
colonnes3 = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)"]
recherche = data[colonnes3].copy()
recherche = recherche.dropna(subset=[colonnes3[1]])

recherche[colonnes3[1]] = recherche[colonnes3[1]].apply(nettoyer_texte)
recherche["Note recherche"] = recherche[colonnes3[1]].apply(predire_note)

print(recherche["Note recherche"].value_counts())

1/1 [==============================] - 0s 36ms/step
0.671572    3
0.699519    1
0.913661    1
0.246065    1
0.347086    1
           ..
0.710241    1
0.840789    1
0.525877    1
0.235803    1
0.759301    1
Name: Note recherche, Length: 183, dtype: int64


In [27]:
colonnes4 = ["SID", "Pouvez-vous expliquer vos réponses merci ..."]
merci = data[colonnes4].copy()
merci = merci.dropna(subset=[colonnes4[1]])

merci[colonnes4[1]] = merci[colonnes4[1]].apply(nettoyer_texte)
merci["Note merci"] = merci[colonnes4[1]].apply(predire_note)

print(merci["Note merci"].value_counts())

1/1 [==============================] - 0s 30ms/step
0.625422    1
0.223015    1
0.813798    1
0.177686    1
0.837797    1
           ..
0.576269    1
0.537384    1
0.453070    1
0.825184    1
0.713682    1
Name: Note merci, Length: 366, dtype: int64


In [28]:
nouvelle_df = data[["SID"]].copy()
nouvelle_df["Totalite des commentaires"] = data[texte_colonnes].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
nouvelle_df = nouvelle_df.dropna(subset=["Totalite des commentaires"])
nouvelle_df = nouvelle_df[nouvelle_df["Totalite des commentaires"].apply(lambda x: x.strip() != "")]

print(nouvelle_df.head())
nouvelle_df["Totalite des commentaires"] = nouvelle_df["Totalite des commentaires"].apply(nettoyer_texte)
nouvelle_df["Note totalite"] = nouvelle_df["Totalite des commentaires"].apply(predire_note)

print(nouvelle_df["Note totalite"].value_counts())

        SID                          Totalite des commentaires
1  15896547  Il n'y a plus d'UFR à l'Université de Bordeaux...
2  15896551  Les conditions de travail dépendent beaucoup d...
4  15896653  L'enseignement est de plus en plus difficile. ...
5  15896801  Je suis dans une équipe Inria. Cela offre des ...
7  15897130  Aujourd’hui, l’informatique offre une multitud...
1/1 [==============================] - 0s 32ms/step
0.671572    2
0.720357    1
0.306281    1
0.101544    1
0.665561    1
           ..
0.476188    1
0.937710    1
0.412562    1
0.346518    1
0.707298    1
Name: Note totalite, Length: 408, dtype: int64


In [29]:
merged_df = pd.merge(personnelle, enseignement, on='SID', how='outer')
merged_df = pd.merge(merged_df, recherche, on='SID', how='outer')
merged_df = pd.merge(merged_df, merci, on='SID', how='outer')
merged_df = pd.merge(merged_df, nouvelle_df, on='SID', how='outer')

In [30]:
merged_df.head(15)

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Note personnelle,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Note recherche,Pouvez-vous expliquer vos réponses merci ...,Note merci,Totalite des commentaires,Note totalite
0,15896547,il n y a plus d ufr à l université de bordeaux...,0.720357,NaN,NaN,NaN,NaN,NaN,NaN,il n y a plus d ufr à l université de bordeaux...,0.720357
1,15897993,temps moyen hors période bouchon,0.753010,a ma connaissance mon enseignement n est pas é...,0.749795,auparavant au cea r d migration université pou...,0.854242,profession en perte de considération dans la s...,0.845736,temps moyen hors période bouchon a ma connaiss...,0.801372
2,15899015,temps moyen de déplacement selon les sites peu...,0.491092,dernière question s il y a et non pas s il n y...,0.581034,NaN,NaN,on peut faire notre recherche comme on le souh...,0.866353,temps moyen de déplacement selon les sites peu...,0.820865
3,15899810,je suis responsable de m mon travail a une for...,0.785849,quand un jeune me demande si devenir prof de f...,0.096319,j ai arrêté pour les raisons indiquées dans mo...,0.310006,non sans hésitation aux trois questions voir m...,0.143254,je suis responsable de m mon travail a une for...,0.157955
4,15901382,pas de distance en temps pour transiter du lie...,0.431251,NaN,NaN,j aime toujours faire de la recherche mais j a...,0.893504,métier intéressant car très varié et avec des ...,0.445727,pas de distance en temps pour transiter du lie...,0.374583
5,15902826,ufr d informatique propre composée à d informa...,0.777198,NaN,NaN,NaN,NaN,admin car la plupart du temps c est de l admin...,0.641630,ufr d informatique propre composée à d informa...,0.659622
6,15903001,je ne fais pratiquement plus ma recherche dans...,0.714753,NaN,NaN,NaN,NaN,la charge administrative que j ai choisie n ét...,0.759924,je ne fais pratiquement plus ma recherche dans...,0.747883
7,15903160,enseignement sur plusieurs sites de la ville l...,0.546957,NaN,NaN,NaN,NaN,la liberté intellectuelle apportée par le vole...,0.206843,enseignement sur plusieurs sites de la ville l...,0.492730
8,15903369,une partie de l enseignement sur le même lieu ...,0.441924,la très forte augmentation du nombre d étudian...,0.533071,le manque de temps à consacrer à la recherche ...,0.608588,depuis ans je n ai pas eu le temps de déposer ...,0.319152,une partie de l enseignement sur le même lieu ...,0.296887
9,15904052,sans hdr par choix de ne pas rédiger ce docume...,0.578332,NaN,NaN,NaN,NaN,le métier se dégrade nous faisons de plus en p...,0.580963,sans hdr par choix de ne pas rédiger ce docume...,0.529365


In [31]:
merged_df.columns

Index(['SID',
       'Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)',
       'Note personnelle',
       'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)',
       'Note enseignement',
       'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)',
       'Note recherche', 'Pouvez-vous expliquer vos réponses merci ...',
       'Note merci', 'Totalite des commentaires', 'Note totalite'],
      dtype='object')

In [32]:
merged_df.to_csv("resultat_analyse_sentiments_float_final.csv", index=False)

In [33]:
personnelle.to_csv("personnelle_float_final.csv", index=False)
enseignement.to_csv("enseignement_float_final.csv", index=False)
recherche.to_csv("recherche_float_final.csv", index=False)
merci.to_csv("merci_float_final.csv", index=False)
nouvelle_df.to_csv("totalite_float_final.csv", index=False)